In [1]:
# main things we need to do:
# 0.⁠ ⁠prepare (collect data and clean) auth df - V
# 0.⁠ ⁠prepare (collect data and clean) events_df - V
# 0.⁠ ⁠preprocess auth df - fill null values - V
# 1.⁠ ⁠run matching (match_and_plot) - V
# 2.⁠ ⁠⁠check plots of different distance metrics results - V (decision = cosine, k=3...10)
# 3.⁠ ⁠⁠run diff_in_diff analysis for target authorities and each event (conviction etc.) (calc_diff_in_diff) - 
# 4.⁠ ⁠⁠check plots of DiD (compare_pred_true)
# 5.⁠ ⁠⁠check if we see the same results across different events (conviction etc.)
# 5.⁠ ⁠⁠check hyper params plot of DiD (plot_across_hyper_params)
# 6.⁠ ⁠⁠run diff_in_diff analysis for placebo auth - sample some from the control group, sample year for treatment and run the same piepline
# 7.⁠ ⁠⁠then - check that we dont see a major diff in diff there
# 8.⁠ ⁠⁠summarize results
# 9.⁠ ⁠⁠conclude in report

In [4]:
import pandas as pd
from pathlib import Path
src_path = Path.cwd().parents[0]

df_matching = pd.read_csv(src_path / 'data/matching_features_df.csv')
df_matching = df_matching.rename(columns={'authority_code': 'auth_id', 'authority_name': 'auth_name'})
df_treatment = df_matching[df_matching['is_treatment'] == 1]
df_control = df_matching[df_matching['is_treatment'] == 0]
df_matching

,year,auth_id,auth_name,is_treatment,district,distance_from_tel_aviv,year_of_municipal_status,council_members_number,socioeconomic_level,compactness_cluster,...,regular_budget_revenues,gov_participation,gov_participation_change,special_budget_revenues,property_tax_total_prev_year,property_tax_residential_prev_year,property_tax_business_prev_year,property_tax_total_current_year,property_tax_residential_current_year,property_tax_business_current_year
0,1999,26,ראש פינה,0,4.0,134.378618,1949.0,7.0,9.0,7.0,...,16192.0,4056.0,13.500,6813.0,9.150079e+04,19540.523333,2183.253333,41378.441674,8802.270211,2012.841105
1,2000,26,ראש פינה,0,4.0,134.378618,1949.0,7.0,7.0,7.0,...,15810.0,3366.0,-17.900,4927.0,9.150079e+04,19540.523333,2183.253333,41378.441674,8802.270211,2012.841105
2,2001,26,ראש פינה,0,4.0,134.378618,1949.0,7.0,7.0,7.0,...,16018.0,3569.0,4.900,5687.0,9.150079e+04,19540.523333,2183.253333,41378.441674,8802.270211,2012.841105
3,2002,26,ראש פינה,0,4.0,134.378618,1949.0,7.0,7.0,7.0,...,17902.0,3239.0,-2.700,3380.0,9.150079e+04,19540.523333,2183.253333,41378.441674,8802.270211,2012.841105
4,2003,26,ראש פינה,0,4.0,134.378618,1949.0,7.0,7.0,7.0,...,19165.0,2786.0,-28.700,6901.0,9.150079e+04,19540.523333,2183.253333,41378.441674,8802.270211,2012.841105
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2875,2018,9800,בנימינה-גבעת עדה,0,0.0,44.388835,1950.0,13.0,8.0,10.0,...,97992.0,29670.0,1.200,30304.0,1.780200e+04,545.400000,44.400000,17647.500000,604.900000,49.600000
2876,2019,9800,בנימינה-גבעת עדה,0,0.0,44.388835,1950.0,13.0,9.0,10.0,...,102963.0,30642.0,1.065,19963.0,3.816485e+06,71618.466667,3363.170000,17497.000000,614.000000,49.000000
2877,2020,9800,בנימינה-גבעת עדה,0,0.0,44.388835,1950.0,13.0,9.0,10.0,...,103935.0,36491.0,1.065,13207.0,3.816485e+06,71618.466667,3363.170000,840.000000,628.000000,53.000000
2878,2021,9800,בנימינה-גבעת עדה,0,0.0,44.388835,1950.0,13.0,8.0,10.0,...,111242.0,33234.0,1.065,25954.0,3.816485e+06,71618.466667,3363.170000,862.960000,641.120000,50.580000


# Match

In [ ]:
df_control.groupby(['year'])['auth_id'].count()

In [ ]:
df_treatment

In [ ]:
df_treatment.groupby(['year'])['auth_id'].count()

In [ ]:
treatment_ids = df_treatment['auth_id'].unique()
print(len(treatment_ids), treatment_ids)
control_ids = df_control['auth_id'].unique()
print(len(control_ids), control_ids)

In [ ]:
df_control

In [ ]:
from src.matching.create_vecs import create_vecs
# control_vecs = create_vecs(df_control, method="agg")
control_vecs = create_vecs(df_control, method="concat")
control_vecs

In [ ]:
from src.matching.create_vecs import create_vecs
# treatment_vecs = create_vecs(df_treatment, method="agg")
treatment_vecs = create_vecs(df_treatment, method="concat")
treatment_vecs

In [ ]:
concat_matching_cols = treatment_vecs.columns.difference(['auth_id', 'auth_name', 'is_treatment']).to_list()
len(concat_matching_cols), concat_matching_cols

In [ ]:
matching_cols = ['district',
    'distance_from_tel_aviv', 'year_of_municipal_status',
    'agg_council_members_number', 'agg_socioeconomic_level',
    'agg_compactness_cluster', 'agg_peripheral_cluster',
    'agg_total_jurisdiction_area', 'agg_population_density',
    'agg_total_population', 'agg_jews_and_others', 'agg_total_men',
    'agg_total_women', 'agg_dependency_ratio',
    'agg_natural_increase_per_1000', 'agg_total_migration_balance',
    'agg_immigrants_post_1990', 'agg_population_growth_rate',
    'agg_residential_percent_area', 'agg_education_percent_area',
    'agg_health_welfare_percent_area', 'agg_public_services_percent_area',
    'agg_culture_leisure_sports_percent_area', 'agg_commerce_percent_area',
    'agg_industry_infrastructure_percent_area',
    'agg_transportation_percent_area',
    'agg_agricultural_buildings_percent_area',
    'agg_public_open_space_percent_area',
    'agg_forest_woodland_percent_area',
    'agg_orchards_citrus_olive_percent_area',
    'agg_cultivated_fields_percent_area',
    'agg_other_open_space_percent_area', 'agg_total_classes',
    'agg_total_students', 'agg_avg_students_per_class',
    'agg_percent_bagrut_eligible', 'agg_percent_university_bagrut',
    'agg_water_receipts', 'agg_total_private_cars', 'agg_avg_car_age',
    'agg_accidents_per_1000_residents', 'agg_accidents_per_1000_vehicles',
    'agg_unemployment_benefits_total', 'agg_male_unemployment_benefits',
    'agg_avg_age_unemployment_recipients',
    'agg_income_support_recipients_yearly', 'agg_avg_salary_employees',
    'agg_avg_salary_male', 'agg_avg_salary_female',
    'agg_change_salary_emp_yoy', 'agg_change_salary_male_yoy',
    'agg_change_salary_female_yoy', 'agg_employees_total',
    'agg_percent_earning_min_wage', 'agg_self_employed_total',
    'agg_avg_income_self_employed', 'agg_change_income_self_employed_yoy',
    'agg_percent_earning_half_avg_salary', 'agg_inequality_index_gini',
    'agg_total_revenues', 'agg_revenues_change',
    'agg_regular_budget_revenues', 'agg_gov_participation',
    'agg_gov_participation_change', 'agg_special_budget_revenues',
    'agg_property_tax_total_prev_year',
    'agg_property_tax_residential_prev_year',
    'agg_property_tax_business_prev_year',
    'agg_property_tax_total_current_year',
    'agg_property_tax_residential_current_year',
    'agg_property_tax_business_current_year']
len(matching_cols)

In [ ]:
from src.matching.matching_funcs import match_and_plot
results_dir_knn = src_path / 'results' / 'knn'

k = 10
metric='cosine' # 'euclidean' or 'cosine' or 'manhattan' or 'minkowski' or 'chebyshev' or 'hamming' or 'jaccard'
match_and_plot(treatment_vecs, control_vecs, concat_matching_cols, k, metric, results_dir_knn)

# Infer

In [5]:
infer_df = pd.read_csv(src_path / 'data/inference_features_df.csv')
infer_df = infer_df.rename(columns={'authority_code': 'auth_id'})
infer_df = infer_df.merge(df_matching[['auth_id', 'auth_name']], on='auth_id', how='left')
infer_df

,year,is_treatment,incident_year,incident_type,auth_id,total_in_migration,male_in_migration,female_in_migration,in_migration_age_15_29,total_out_migration,...,total_expenditures,expenditures_change,operations_expenditures,loan_repayment_expenditures,financing_expenditures,special_budget_expenditures,budget_surplus_deficit,annual_surplus_deficit,cumulative_deficit,auth_name
0,1999,0,NaN,NaN,26,127.0,67.0,60.0,41.0,87.0,...,22876.0,29.5,8470.0,NaN,129.0,NaN,261.0,261.00,NaN,ראש פינה
1,1999,0,NaN,NaN,26,127.0,67.0,60.0,41.0,87.0,...,22876.0,29.5,8470.0,NaN,129.0,NaN,261.0,261.00,NaN,ראש פינה
2,1999,0,NaN,NaN,26,127.0,67.0,60.0,41.0,87.0,...,22876.0,29.5,8470.0,NaN,129.0,NaN,261.0,261.00,NaN,ראש פינה
3,1999,0,NaN,NaN,26,127.0,67.0,60.0,41.0,87.0,...,22876.0,29.5,8470.0,NaN,129.0,NaN,261.0,261.00,NaN,ראש פינה
4,1999,0,NaN,NaN,26,127.0,67.0,60.0,41.0,87.0,...,22876.0,29.5,8470.0,NaN,129.0,NaN,261.0,261.00,NaN,ראש פינה
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
78907,2022,0,NaN,NaN,9800,617.0,331.0,286.0,84.0,536.0,...,130811.0,NaN,43052.0,4786.0,918.0,16634.0,50.0,0.05,-10.451,בנימינה-גבעת עדה
78908,2022,0,NaN,NaN,9800,617.0,331.0,286.0,84.0,536.0,...,130811.0,NaN,43052.0,4786.0,918.0,16634.0,50.0,0.05,-10.451,בנימינה-גבעת עדה
78909,2022,0,NaN,NaN,9800,617.0,331.0,286.0,84.0,536.0,...,130811.0,NaN,43052.0,4786.0,918.0,16634.0,50.0,0.05,-10.451,בנימינה-גבעת עדה
78910,2022,0,NaN,NaN,9800,617.0,331.0,286.0,84.0,536.0,...,130811.0,NaN,43052.0,4786.0,918.0,16634.0,50.0,0.05,-10.451,בנימינה-גבעת עדה


In [6]:
infer_df.columns

Index(['year', 'is_treatment', 'incident_year', 'incident_type', 'auth_id',
       'total_in_migration', 'male_in_migration', 'female_in_migration',
       'in_migration_age_15_29', 'total_out_migration', 'male_out_migration',
       'female_out_migration', 'out_migration_age_15_29',
       'start_roads_length_km', 'start_roads_area_sqm', 'start_water_pipes_km',
       'start_sewage_pipes_km', 'start_drainage_pipes_km',
       'complete_roads_length_km', 'complete_roads_area_sqm',
       'complete_water_pipes_km', 'complete_sewage_pipes_km',
       'complete_drainage_pipes_km', 'satisfaction_municipal_services',
       'satisfaction_municipal_performance', 'satisfaction_residential_area',
       'satisfaction_cleanliness', 'satisfaction_parks_green_spaces',
       'feeling_safe_dark', 'contacting_local_authority', 'total_expenditures',
       'expenditures_change', 'operations_expenditures',
       'loan_repayment_expenditures', 'financing_expenditures',
       'special_budget_expendit

In [8]:
infer_df[infer_df['is_treatment'] == 1].groupby('auth_id')['incident_type'].nunique()

auth_id
1034    2
2630    1
2650    2
3000    3
6200    3
6500    1
6700    3
6900    3
7000    3
7100    3
8000    1
8400    1
8600    2
9100    1
Name: incident_type, dtype: int64

In [10]:
infer_df[infer_df['auth_id']==7000][['auth_name', 'year', 'incident_year', 'incident_type']].drop_duplicates()

,auth_name,year,incident_year,incident_type
62208,לוד,1999,2006.0,arrest
62232,לוד,1999,2006.0,indictment filed
62256,לוד,1999,2010.0,conviction
62280,לוד,2000,2006.0,arrest
62304,לוד,2000,2006.0,indictment filed
...,...,...,...,...
63816,לוד,2021,2006.0,indictment filed
63840,לוד,2021,2010.0,conviction
63864,לוד,2022,2006.0,arrest
63888,לוד,2022,2006.0,indictment filed


In [11]:
columns_to_predict = [
    'total_in_migration', 'male_in_migration', 'female_in_migration',
    'in_migration_age_15_29', 'total_out_migration', 'male_out_migration',
    'female_out_migration', 'out_migration_age_15_29',
    'start_roads_length_km', 'start_roads_area_sqm', 'start_water_pipes_km',
    'start_sewage_pipes_km', 'start_drainage_pipes_km',
    'complete_roads_length_km', 'complete_roads_area_sqm',
    'complete_water_pipes_km', 'complete_sewage_pipes_km',
    'complete_drainage_pipes_km', 'satisfaction_municipal_services',
    'satisfaction_municipal_performance', 'satisfaction_residential_area',
    'satisfaction_cleanliness', 'satisfaction_parks_green_spaces',
    'feeling_safe_dark', 'contacting_local_authority', 'total_expenditures',
    'expenditures_change', 'operations_expenditures',
    'loan_repayment_expenditures', 'financing_expenditures',
    'special_budget_expenditures', 'budget_surplus_deficit',
    'annual_surplus_deficit', 'cumulative_deficit']

In [13]:
from src.diff_in_diff.diff_in_diff_funcs import calc_diff_in_diff
results_dir = src_path / 'results'

infer_df.set_index('auth_id', inplace=True)

calc_diff_in_diff(
    infer_df,
    columns_to_predict=columns_to_predict, 
    treatment_ids=df_treatment['auth_id'].tolist(),
    k=10,
    distance_metric='cosine',
    results_dir=results_dir,
    diff_results_dir=(results_dir / 'diff'),
)

ValueError: No objects to concatenate